In [1]:
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from ai import get_team_code
from functools import  partial

## Load and format data

In [2]:
X = pd.read_csv('../data/processed/epl.csv')
X = X[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "Date", "Time"]]
X = X.rename(columns={"HomeTeam": "home", "AwayTeam": "away", "FTHG": "home_goals", "FTAG": "away_goals", "Date": "date", "Time": "time"})
X['date'] = pd.to_datetime(X['date'])
X = X.dropna(how='all')  # Drop rows with all null values
X['time'] = X['time'].fillna(method='bfill').fillna(method='ffill')
X['time'] = pd.to_datetime(X['time'], format='%H:%M').dt.time
X = X.sort_values(by=['date'])


team_code = partial(get_team_code, 'epl')
X['home'] = X['home'].apply(team_code)
X['away'] = X['away'].apply(team_code)

/home/kevon/src/freekick/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (99) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
X.head()

,home,away,home_goals,away_goals,date,time
8004,BLA,ARS,1.0,1.0,1993-01-09,16:00:00
8010,TOT,CHE,1.0,1.0,1993-01-09,16:00:00
8009,SWI,MCI,1.0,3.0,1993-01-09,16:00:00
8008,SHW,NOR,3.0,3.0,1993-01-09,16:00:00
8007,QPR,SHU,2.0,1.0,1993-01-09,16:00:00


In [6]:
X.dtypes

home                  object
away                  object
home_goals           float64
away_goals           float64
date          datetime64[ns]
time                  object
dtype: object

In [7]:
X.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10044 entries, 8004 to 5082
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   home        10044 non-null  object        
 1   away        10044 non-null  object        
 2   home_goals  10044 non-null  float64       
 3   away_goals  10044 non-null  float64       
 4   date        10044 non-null  datetime64[ns]
 5   time        10044 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 549.3+ KB


In [8]:
team_rank_df = pd.read_csv('../data/processed/epl_team_ranking.csv', index_col=['date', 'ranking'], parse_dates=True)
team_rank_df.columns = [team_code(c) for c in team_rank_df.columns]
team_rank_df = team_rank_df.sort_values(by=['date'])

In [9]:
team_rank_df.head()

MCI   LIV   CHE   TOT   ARS   MUN   WHU   WOL   BHA  \
date       ranking                                                         
2022-01-03 defense   3.0   3.0   2.4   2.2   2.2   2.2   2.0   1.7   1.9   
           offense   0.2   0.3   0.3   0.7   0.5   0.6   0.7   0.6   0.6   
           overall  93.8  92.9  88.9  78.0  82.4  79.3  74.0  73.5  75.1   
           overall  93.7  93.9  88.2  80.7  82.3  77.0  72.0  68.8  77.5   
           offense   0.2   0.2   0.4   0.6   0.5   0.7   0.8   0.7   0.5   

                     LEI   AVL   CRY   SOU   LEE   BRE   EVE   BUR   WAT  \
date       ranking                                                         
2022-01-03 defense   2.2   1.9   1.8   2.0   1.9   1.7   1.8   1.7   1.6   
           offense   1.0   0.7   0.7   0.8   1.1   0.9   0.9   0.8   1.0   
           overall  71.1  73.3  71.2  72.3  62.6  65.6  66.4  66.3  59.3   
           overall  69.6  77.1  75.2  65.0  63.4  71.8  64.3  64.3  57.9   
           offense   0.9   0.6   0.6   1.0   1.1   0.8   0.9   0.9   1.1   

                     NEW   NOR  
date       ranking              
2022-01-03 defense   1.7   1.5  
           offense   0.9   1.1  
           overall  65.8  54.1  
           overall  70.3  52.3  
           offense   0.7   1.2

In [115]:
team_rank_cp = team_rank_df.copy()
# team_rank_cp = team_rank_cp.reset_index()
# team_rank_cp = team_rank_cp.set_index(['date', 'ranking'])

In [116]:
team_rank_cp.head()

MCI   LIV   CHE   TOT   ARS   MUN   WHU   WOL   BHA  \
date       ranking                                                         
2022-01-03 defense   3.0   3.0   2.4   2.2   2.2   2.2   2.0   1.7   1.9   
           defense   2.9   2.9   2.5   2.3   2.2   2.1   1.9   1.7   1.9   
           offense   0.2   0.2   0.4   0.6   0.5   0.7   0.8   0.7   0.5   
           overall  93.7  93.9  88.2  80.7  82.3  77.0  72.0  68.8  77.5   
           offense   0.2   0.3   0.3   0.7   0.5   0.6   0.7   0.6   0.6   

                     LEI   AVL   CRY   SOU   LEE   BRE   EVE   BUR   WAT  \
date       ranking                                                         
2022-01-03 defense   2.2   1.9   1.8   2.0   1.9   1.7   1.8   1.7   1.6   
           defense   1.9   2.0   1.9   1.9   1.9   1.9   1.7   1.7   1.6   
           offense   0.9   0.6   0.6   1.0   1.1   0.8   0.9   0.9   1.1   
           overall  69.6  77.1  75.2  65.0  63.4  71.8  64.3  64.3  57.9   
           offense   1.0   0.7   0.7   0.8   1.1   0.9   0.9   0.8   1.0   

                     NEW   NOR  
date       ranking              
2022-01-03 defense   1.7   1.5  
           defense   1.8   1.5  
           offense   0.7   1.2  
           overall  70.3  52.3  
           offense   0.9   1.1

In [110]:
pd.merge_asof(X, team_rank_cp,on='date', by='home')

,home,away,home_goals,away_goals,date,time
0,ARS,CRY,2.0,1.0,2014-08-16,12:30:00
1,LEI,EVE,2.0,2.0,2014-08-16,12:30:00
2,MUN,SWA,1.0,2.0,2014-08-16,12:30:00
3,QPR,HUL,0.0,1.0,2014-08-16,12:30:00
4,STO,AVL,0.0,1.0,2014-08-16,12:30:00
...,...,...,...,...,...,...
10736,SWA,FUL,0.0,3.0,2013-05-19,16:00:00
10737,TOT,SUN,1.0,0.0,2013-05-19,16:00:00
10738,WBA,MUN,5.0,5.0,2013-05-19,16:00:00
10739,WHU,REA,4.0,2.0,2013-05-19,16:00:00


In [6]:
X.describe()

,home_goals,away_goals
count,10044.000000,10044.000000
mean,1.520410,1.138690
std,1.302301,1.135884
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,9.000000,9.000000


In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10044 entries, 0 to 10740
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   home        10044 non-null  object        
 1   away        10044 non-null  object        
 2   home_goals  10044 non-null  float64       
 3   away_goals  10044 non-null  float64       
 4   date        10044 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 470.8+ KB


In [8]:
X.dtypes

home                  object
away                  object
home_goals           float64
away_goals           float64
date          datetime64[ns]
dtype: object

In [9]:
X.shape

(10044, 5)

In [10]:
X.head()

,home,away,home_goals,away_goals,date
0,ARS,CRY,2.0,1.0,2014-08-16
1,LEI,EVE,2.0,2.0,2014-08-16
2,MUN,SWA,1.0,2.0,2014-08-16
3,QPR,HUL,0.0,1.0,2014-08-16
4,STO,AVL,0.0,1.0,2014-08-16


In [11]:
y = np.sign(X['home_goals'] - X['away_goals'])

In [12]:
y.head()

0    1.0
1    0.0
2   -1.0
3   -1.0
4   -1.0
dtype: float64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
X_train.head()

,home,away,home_goals,away_goals,date
4590,LIV,LEI,1.0,0.0,2002-01-30
9611,CRY,CAR,0.0,0.0,2018-12-26
4821,CRY,NEW,0.0,2.0,2020-11-27
5309,TOT,EVE,4.0,1.0,1998-12-28
3601,CHE,POR,4.0,0.0,2008-08-17


In [15]:
pd.get_dummies(X, columns=['home','away'],prefix=['home_', 'away_']).head()

,home_goals,away_goals,date,home__ARS,home__AVL,home__BAR,home__BHA,home__BIR,home__BLA,home__BLP,...,away__SUN,away__SWA,away__SWI,away__TOT,away__WAT,away__WBA,away__WHU,away__WIG,away__WIM,away__WOL
0,2.0,1.0,2014-08-16,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,2.0,2014-08-16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2.0,2014-08-16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0.0,1.0,2014-08-16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,1.0,2014-08-16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## KNN Model Analysis

In [16]:
# from sklearn.neighbors import KNeighborsClassifier
# import matplotlib.pyplot as plt

# training_accuracy = []
# test_accuracy = []

# # try neighbors from 1 to 10
# neighbors_list = range(1,11)

# for n_neighbors in neighbors_list:
#     # build model
#     clf = KNeighborsClassifier(n_neighbors=n_neighbors)
#     clf.fit(X_train, y_train)
#     # Record training set accuracy
#     training_accuracy.append(clf.score(X_train, y_train))
#     # record generalized accuracy
#     test_accuracy.append(clf.score(X_test, y_test))
    
# plt.plot(neighbors_list, training_accuracy, label="training accuracy")
# plt.plot(neighbors_list, test_accuracy, label="test accuracy")
# plt.ylabel("Accuracy")
# plt.xlabel('n_neighbors')
# plt.legend()
# plt.show()

In [12]:
raw_df = pd.read_csv('../data/raw/epl/season_1993-1994.csv')
raw_df2 = pd.read_csv('../data/raw/epl/season_2021-2022.csv')

In [16]:
join_df = pd.concat([raw_df, raw_df2], axis=0)

In [17]:
join_df.shape

(836, 127)

In [22]:
join_df['Time']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
       ...  
279    16:30
280    20:00
281    19:30
282    20:15
283    19:45
Name: Time, Length: 836, dtype: object

In [24]:
join_df['Time'].fillna(method='bfill').isna().sum()

0